
# QuantumCircuit — Demo

This notebook walks through the main features of the lightweight `QuantumCircuit` class we provided in `quantum_circuit.py`:

- Build circuits and append gates (single- and two-qubit)
- Add measurements and resets
- Draw an ASCII diagram
- Export to OpenQASM 2 and 3 (in a compatible form with specific SDKs like Qiskit/Braket/Tket)
- Customize QASM emission with templates and float precision
- Calculate basic circuit properties like gate counts and depth



In [ ]:
from benchmarks import QasmEmitter


In [ ]:
from benchmarks import QuantumCircuit, QasmEmitterOptions


## 1) Quick start: a Bell state

We'll create a 2‑qubit circuit, put qubit 0 into superposition with `H`, then entangle with `CX`. We'll also add measurements.


In [ ]:

qc = QuantumCircuit(number_of_qubits=2, number_of_classical_bits=2)
qc.add_h_gate(0)
qc.add_cx_gate(control_qubit=0, target_qubit=1)
qc.add_measurement([0,1], [0,1])   # measure both qubits into bits 0,1
print(qc)                           # uses __repr__ summary



## 2) Draw an ASCII circuit diagram

The `draw_circuit_diagram` helper prints a simple text diagram. UUse `max_length` if your circuit is too long to chunk the output.


In [ ]:

qc.draw_circuit_diagram()



## 3) Export to OpenQASM 2 (Qiskit-style)

Use `to_qasm()` with `format="qasm2"` and `target_sdk="qiskit"` to get gate names and includes appropriate for Qiskit.


In [ ]:

qasm2 = qc.to_qasm(format="qasm2", target_sdk="qiskit")
print(qasm2)



## 4) Export to OpenQASM 3 (Braket‑style)

OpenQASM 3 is supported; for AWS Braket, set `target_sdk="braket"`. (Note: Braket uses `cnot` for `cx` and `U` for a `u` gate.)


In [ ]:

qasm3 = qc.to_qasm(format="qasm3", target_sdk="braket")
print(qasm3)



## 5) Gate counts & helpers

You can query counts of single-qubit vs two-qubit gates, and also print a built-in list of supported gates.


In [ ]:

print("Single-qubit gate count:", qc.single_qubit_gate_count())
print("Two-qubit gate count:", qc.two_qubit_gate_count())

print("\nSupported gates:")
qc.display_gate_descriptions()#the list of suported gates will be extended in the future if it is requered 



## 6) Parameterized rotations and `u(θ,φ,λ)`

Let's add some parameterized single‑qubit gates, then export with custom float precision.


In [ ]:

qc2 = QuantumCircuit(number_of_qubits=1, number_of_classical_bits=1)
qc2.add_rx_gate(0, 0.123456789)
qc2.add_ry_gate(0, 1.23456789)
qc2.add_rz_gate(0, 2.3456789)
qc2.add_u_gate(0, theta=0.3333333333, phi=0.6666666666, lambda_parameter=1.23456789)
qc2.add_measurement(0,0)

# Default float precision is 6
print("QASM2 default precision=6:")
print(qc2.to_qasm(format="qasm2"))

# Increase precision to 10
opts = QasmEmitterOptions(format="qasm2", float_precision=10)
print("\nQASM2 precision=10:")
print(qc2.to_qasm(emitter_options=opts))



## 7) Two‑qubit gates: `CY`, `CZ`, and `SWAP`

`TwoQubitQuantumGate` is used internally when you pass a control qubit. `SWAP` is modeled as a two‑qubit gate as well.


In [ ]:

qc3 = QuantumCircuit(number_of_qubits=3, number_of_classical_bits=3)
qc3.add_h_gate(0)
qc3.add_cy_gate(control_qubit=0, target_qubit=2)
qc3.add_cz_gate(control_qubit=2, target_qubit=1)
qc3.add_swap_gate(0, 1)
qc3.add_measurement([0,1,2], [0,1,2])

print(qc3)
qc3.draw_circuit_diagram()
print(qc3.to_qasm(format="qasm2"))



## 8) Resets

You can insert explicit `reset` operations for qubits.


In [ ]:

qc4 = QuantumCircuit(number_of_qubits=2, number_of_classical_bits=2)
qc4.add_x_gate(0)
qc4.add_reset_operation(0)
qc4.add_h_gate(0)
qc4.add_cx_gate(control_qubit=0, target_qubit=1)
qc4.add_measurement([0,1], [0,1])

qc4.draw_circuit_diagram()
print(qc4.to_qasm(format="qasm2"))



## 9) SDK aliasing and includes

The emitter supports multiple SDKs. The predefined ones are:

- **QASM 2 - Qiskit**
- **QASM 3 - Qiskit**
- **QASM 2 + PyTket** 
- **QASM 3 + Braket**

But, you can also pass a custom mapping via `custom_template` using the `QasmEmitterOptions` configuration class.

In [ ]:
# QASM2 + Qiskit
print("QASM 2 - Qiskit ")
print(qc2.to_qasm(format="qasm2", target_sdk="qiskit"))

# QASM3 + Braket
print("\nQASM 3 - Braket")
print(qc2.to_qasm(format="qasm3", target_sdk="braket"))

# Custom template: override internal 'rz' to emit as 'phase' (toy example)
opts_custom = QasmEmitterOptions(format="qasm3", target_sdk="custom",
                                 custom_template={"rz": "phase"})
print("\nQASM 3 + custom 'rz'→'phase':")
print(qc2.to_qasm(emitter_options=opts_custom))
